In [2]:
%store -r summary_data_dir_activity
%store -r activity_all_file


%store -r summary_data_dir_person
%store -r persons_all_file

%store -r events_nonmoma

%store -r json_suffix

%store -r linked_data_filepath_group

%store -r datavis_venue_exhibitions 


%store -r linked_data_filepath_activity
%store -r linked_data_filepath_person

In [3]:
try:
    import json
except:
    %pip install json
    import json

from operator import itemgetter

import os

import os

try:
    import urllib
except:
    %pip install urllib
    import urllib

try:
    import hashlib
except:
    %pip install hashlib
    import hashlib

# Linked Art - Create Summary Data Files - Person

This notebook will create summary data files for the MOMA alternative data transformed to Linked Art JSON-LD, for use with an exhibition browser.

#  Create JSON file for all *Person* entities
The following code will create a JSON file 
`persons_all.json`

The JSON file contains summary information about each group entity person data directory:
- id
- _label
- name
- born
- died
- nationality
- identified_by
- referred_to_by
- equivalent
- total_exhibitions
- exhibitions
- gender

# Create summary file for all persons `persons_all.json`

In [12]:


persons = []

person_template = {
    "id": "",
    "_label":"",
    "name":"",
    "born": "",
    "died": "",
    "nationality": "",
    "identified_by": [],
   
    "equivalent": [],
    "total_exhibitions": "",
    "exhibitions": [],
    "gender": "",
    "bio": ""
}

  

for filename in os.listdir(linked_data_filepath_person):
    try:
        with open(os.path.join(linked_data_filepath_person, filename), 'r') as json_file:
            
            data = json.load(json_file)
            this = person_template.copy()

            #id
            this["id"] = data.get("id")
            #_label
            this["_label"] = data.get("_label")

            #name
            for entry in data.get("identified_by"):
                if entry.get("_label") == 'AlphaSortName':
                    this["name"] = entry.get("content")

            #born
            if "born" in data:
                this["born"] = data.get("born").get("timespan").get("begin_of_the_begin").split("T")[0]

            #died
            if "died" in data:
                this["died"] = data.get("died").get("timespan").get("end_of_the_end").split("T")[0]

            # equivalent
            if "equivalent" in data:
                eq_list = []
                for ident in data.get("equivalent"):
                    eq_list.append(ident.get("id"))
                this["equivalent"] = eq_list

            # classified_as
            if "classified_as" in data:

                # gender
                for entry in data.get("classified_as"):
                    if entry.get("id") in["http://vocab.getty.edu/aat/300189557", "http://vocab.getty.edu/aat/300189559"]:
                        this["gender"] = entry.get("_label")
                        
                    # nationality
                    else:
                        this["nationality"] = entry.get("_label")
                        

            # bio
            if "referred_to_by" in data:
                this["bio"] = data.get("referred_to_by")[0].get("content")


            
            copy = this.copy()
            persons.append(copy) 
            
    except:
        continue

with open(os.path.join(summary_data_dir_person, persons_all_file), 'w') as file:
     file.write(json.dumps({"persons": persons},indent=2))

# Add total exhibitions and exhibition summary data, to `persons_all.json`

Add total number of exhibitions per artist to `output/summary/person/persons_all.json`

Method:
- iterate over persons in `persons_all.json`
  - get person_id
  - iterate over exhibitions in `events_all.json`
    - if person_id in exhibition's influenced_by list
      - add exhibition to new list for person
        - update person record in `persons_all.json`
          - append list of exhibitions that person influenced
          - append total number of exhibitions that person influenced 


In [13]:
# this list will hold updated person records to overwrite persons_all.json
updated_persons_list = []

# get events summary data
with open(os.path.join(summary_data_dir_activity, activity_all_file), 'r') as json_file:
    exdata = json.load(json_file)

# open summary file api/persons_all.json
with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:
    data = json.load(json_file)

    #iterate through each person in persons_all.json
    for person in data["persons"]:
        # get id
        person_id = person.get("id")
        
        person_exhibitions = []

        # iterate through each exhibition in events_all.json
        for exhibition in exdata.get('events'):
            
            # if person's id is in the exhibition's 'influenced_by' list, append the exhibition info to the person summary entry in persons_all.json
             
            for person_ex in exhibition.get("influenced_by"):
                
                if person_ex.get("id").lower() == person_id:
                    

                    # append exhibition summary info to person_exhibitions[] list
                    id = exhibition.get("id")
                    label = exhibition.get("_label")
                    start = exhibition.get("start")
                    end = exhibition.get("end")

                    org = exhibition.get("org")
                    location = exhibition.get("location")
                    person_exhibitions.append({"id":id,"_label": label, "start": start, "end": end, "org": org, "location": location})
                    break
                    
                
        
        # calculate total number of exhibitions for person by counting number of exhibitions for person
        total_exhibitions = len(person_exhibitions)

        # copy existing person entry in persons_all.json and add new properties
        updated_person = person.copy()
        updated_person["total_exhibitions"] = total_exhibitions
        updated_person["exhibitions"] = person_exhibitions
        updated_persons_list.append(updated_person)

    with open(os.path.join(summary_data_dir_person, persons_all_file), 'w') as file:
        file.write(json.dumps({"persons": updated_persons_list},indent=2))
    
       

## Create summary file `persons_all_surname.json` with persons ordered by first letter of surname and then first three letters of surname

In [ ]:
dict_surname = {}

with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    field = 'name'
    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(field) != "":
            list.append(entry)
    
    # order list by name
    sorted_list = sorted(list, key=itemgetter(field), reverse=False) 
    
    for person in sorted_list:
        name = person.get(field)
        first_letter = name[:1].upper()
        first_three_letters = name[:3]

        if first_letter not in dict_surname:
            dict_surname[first_letter] = {}
            dict_surname[first_letter][first_three_letters] = []

        if first_three_letters not in dict_surname[first_letter]:
            dict_surname[first_letter][first_three_letters] = []

        dict_surname[first_letter][first_three_letters].append(person)

        
    
    # write to file 
    with open(os.path.join(summary_data_dir_person, 'persons_all_surname.json'), 'w') as file:
        file.write(json.dumps({"persons": dict_surname},indent=2))

## Create persons_all_surname_full.json

Create file with persons ordered by full surname

In [ ]:
dict_surname = {}

with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    field = 'name'
    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(field) != "":
            list.append(entry)
    
    # order list by name
    sorted_list = sorted(list, key=itemgetter(field), reverse=False) 
    
    for person in sorted_list:
        name = person.get(field)
        
        surname = name.split(" ")[0].upper()
        
        if surname not in dict_surname:
            dict_surname[surname] = []
            
        dict_surname[surname].append(person)
  
    
    # write to file 
    with open(os.path.join(summary_data_dir_person, 'persons_all_surname_full.json'), 'w') as file:
        file.write(json.dumps({"persons": dict_surname},indent=2))

In [ ]:
## create list of persons ordered by birthyear and then first letter or surname

dict_year = {}
count_year = {}


with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    born_field = 'born'
    name_field = 'name'

    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(born_field) != "":
            list.append(entry)
    
    # order list by born
    sorted_list = sorted(list, key=itemgetter(born_field), reverse=True) 
    
    for person in sorted_list:

        born = person.get(born_field)
       
        born_year = born.split("-")[0]

        if born_year == "":
            continue

        name = person.get(name_field)
        first_letter = name[:1].upper()
        

        if born_year not in dict_year:
            dict_year[born_year] = {}
            count_year[born_year] = 0
            dict_year[born_year][first_letter] = []

        if first_letter not in dict_year[born_year]:
            dict_year[born_year][first_letter] = []

        dict_year[born_year][first_letter].append(person)
        count_year[born_year] += 1

    for year in dict_year:
        letters = dict_year[year]
        for letter in letters:
            person_list = letters[letter]
            sorted_person_list = sorted(person_list, key=itemgetter('name'), reverse=False) 
            dict_year[year][letter] = sorted_person_list
        
    
    # write to file 
    with open(os.path.join(summary_data_dir_person, 'persons_all_born.json'), 'w') as file:
        file.write(json.dumps({"count": count_year, "persons": dict_year},indent=2))

## create persons_nonus_born.json

In [ ]:
## create list of persons ordered by birthyear and then first letter or surname

dict_year = {}
count_year = {}


with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    born_field = 'born'
    name_field = 'name'

    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(born_field) != "":
            list.append(entry)
    
    # order list by born
    sorted_list = sorted(list, key=itemgetter(born_field), reverse=True) 
    
    for person in sorted_list:
        nationality = person.get("nationality")
        if nationality in ["", "American"]:
            continue
        
        born = person.get(born_field)

       
        born_year = born.split("-")[0]

        if born_year == "":
            continue

        name = person.get(name_field)
        first_letter = name[:1].upper()
        

        if born_year not in dict_year:
            dict_year[born_year] = {}
            count_year[born_year] = 0
            dict_year[born_year][first_letter] = []

        if first_letter not in dict_year[born_year]:
            dict_year[born_year][first_letter] = []

        dict_year[born_year][first_letter].append(person)
        count_year[born_year] += 1

    for year in dict_year:
        letters = dict_year[year]
        for letter in letters:
            person_list = letters[letter]
            sorted_person_list = sorted(person_list, key=itemgetter('name'), reverse=False) 
            dict_year[year][letter] = sorted_person_list
        
    
    # write to file 
    with open(os.path.join(summary_data_dir_person, 'persons_nonus_born.json'), 'w') as file:
        file.write(json.dumps({"count": count_year, "persons": dict_year},indent=2))

## create persons_us_born.json

In [ ]:
## create list of persons ordered by birthyear and then first letter or surname

dict_year = {}
count_year = {}


with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    born_field = 'born'
    name_field = 'name'

    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(born_field) != "":
            list.append(entry)
    
    # order list by born
    sorted_list = sorted(list, key=itemgetter(born_field), reverse=True) 
    
    for person in sorted_list:
        nationality = person.get("nationality")
        if nationality != "American":
            continue
        
        born = person.get(born_field)
        born_year = born.split("-")[0]

        if born_year == "":
            continue

        name = person.get(name_field)
        first_letter = name[:1].upper()
        

        if born_year not in dict_year:
            dict_year[born_year] = {}
            count_year[born_year] = 0
            dict_year[born_year][first_letter] = []

        if first_letter not in dict_year[born_year]:
            dict_year[born_year][first_letter] = []

        dict_year[born_year][first_letter].append(person)
        count_year[born_year] += 1

    for year in dict_year:
        letters = dict_year[year]
        for letter in letters:
            person_list = letters[letter]
            sorted_person_list = sorted(person_list, key=itemgetter('name'), reverse=False) 
            dict_year[year][letter] = sorted_person_list
        
    
    # write to file 
    with open(os.path.join(summary_data_dir_person, 'persons_us_born.json'), 'w') as file:
        file.write(json.dumps({"count": count_year, "persons": dict_year},indent=2))

# create file of persons involved in non-moma events, ordered by birth date

In [4]:
# create list of persons ordered by birthyear and then first letter or surname

dict_year = {}
count_year = {}


with open(os.path.join(summary_data_dir_person, persons_all_file), 'r') as json_file:

    born_field = 'born'
    name_field = 'name'

    data = json.load(json_file)
    list = []
    for idx, entry in enumerate(data.get("persons")):
        if entry.get(born_field) != "":
            list.append(entry)

    # order list by born
    sorted_list = sorted(list, key=itemgetter(born_field), reverse=True)

    for person in sorted_list:

        exhibitions = person.get("exhibitions")

        list_exhibition_locations = [key['location'] for key in exhibitions]

        nonmoma_locations = ["Brata Gallery"
                        "Green Gallery",
                        "Hansa Gallery",
                        "Judson Gallery",
                        "Reuben Gallery",
                        "Tanager Gallery",
                        "The City Gallery", ]
        
        check_exhibition_nonmoma = any(item in list_exhibition_locations for item in nonmoma_locations)
        
        if check_exhibition_nonmoma == False:
            continue

        born = person.get(born_field)
        born_year = born.split("-")[0]
        if born_year == "":
            continue

        name = person.get(name_field)
        first_letter = name[:1].upper()

        if born_year not in dict_year:
            dict_year[born_year] = {}
            count_year[born_year] = 0
            dict_year[born_year][first_letter] = []

        if first_letter not in dict_year[born_year]:
            dict_year[born_year][first_letter] = []

        dict_year[born_year][first_letter].append(person)
        count_year[born_year] += 1

    # sort person lists
    for year in dict_year:
        letters = dict_year[year]
        for letter in letters:
            person_list = letters[letter]
            sorted_person_list = sorted(
                person_list, key=itemgetter('name'), reverse=False)
            dict_year[year][letter] = sorted_person_list

    # write to file
    with open(os.path.join(summary_data_dir_person, 'persons_nonmoma_born.json'), 'w') as file:
        file.write(json.dumps(
            {"count": count_year, "persons": dict_year}, indent=2))
